In [35]:
import torch
from torch.autograd import Variable 
from torch.utils.data import DataLoader, TensorDataset
from torchinfo import summary
import torch.nn as nn
from torch.nn.utils import clip_grad_norm
import torch.nn.functional as F
import numpy as np

In [17]:
# Set seeds
seed = 100
torch.manual_seed(seed)
np.random.seed(seed)

In [30]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length, dropout):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True, dropout=dropout)
        
        self.fc = nn.Linear(hidden_size, num_classes)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        
        output, (h1, c1) = self.lstm(x, (h0, c0))
        h1 = h1.view(-1, self.hidden_size)
        result = self.tanh(h1)
        result = self.fc(result)
        result = self.tanh(result)
        
        return result

In [36]:
epochs = 20

num_classes = 1 # number of output classes
input_size = 4 # number of features: MACD, normalized returns, changepoint location, changepoint severity
hidden_size = 5 # range given: 5, 10, 20, 40, 80, 160
num_layers = 1 # number of lstm layers
seq_length = 1
dropout = 0.1 # range given 0.1-0.5
batch_size = 64 # range given: 64, 128, 256

learning_rate = 0.01 # range given: 10^-4 - 10^-1

max_norm = 0.01 # range is 100, 0, 0.01

In [34]:
model = LSTM(num_classes, input_size, hidden_size, num_layers, seq_length, dropout)
summary(model, input_size=(batch_size,1,4))

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     --                        --
├─LSTM: 1-1                              [64, 1, 5]                220
├─Tanh: 1-2                              [64, 5]                   --
├─Linear: 1-3                            [64, 1]                   6
├─Dropout: 1-4                           --                        --
├─Tanh: 1-5                              [64, 1]                   --
Total params: 226
Trainable params: 226
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [15]:
# Daily normalized returns dataframe not created yet

def sharpe_loss(normalized_returns, output):
    output = Variable(output)
    model_returns = torch.sum(torch.multiply(normalized_returns, output)) 
    loss = -1 * torch.sqrt(252) * torch.mean(model_returns) / torch.std(model_returns)
    return loss

In [37]:
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate) 

In [38]:
def train(train_loader):
    for epoch in range(epochs):
        for x_batch, y_batch in train_loader:
            output = model.forward(x_batch)
            
            optimizer.zero_grad()
            loss = sharpe_loss(normalized_returns, output)
            loss.backward()
            
            clip_grad_norm(model.parameters(), max_norm)
            optimizer.step()
            
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
    return model

In [39]:
def main():
    X_train = torch.stack([torch.from_numpy(np.array(i)) for i in X_train])
    y_train = torch.stack([torch.from_numpy(np.array(i)) for i in y_train])
    
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    
    trained_model = train(train_loader)